In [1]:
import logging
import numpy as np
import tensorflow as tf
from node.base import get_node_function
from node.fix_grid import RKSolver


# for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)


def log_calling(fn):
    
    def test_fn(*args, **kwargs):
        logger.debug(f'calling {fn.__name__}')
        return fn(*args, **kwargs)
    
    return test_fn


@tf.function
def dsigmoid(x):
    return tf.nn.sigmoid(x) * (1 - tf.nn.sigmoid(x))


@tf.function
def inv_sigmoid(x):
    return tf.math.log(x + 1e-8) - tf.math.log(1 - x + 1e-8)


@tf.function
def softmax(x, axis):
    return tf.nn.log_softmax(x, axis)


@tf.function
def softmin(x, axis):
    return -tf.nn.log_softmax(-x, axis)


@tf.function
def softrescale(x, axis):
    max = softmax(x, axis)
    min = softmin(x, axis)
    return (x - min) / (max - min)


@log_calling
@tf.function
def rescale(x, axis):
    max = tf.reduce_max(x, axis, keepdims=True)
    min = tf.reduce_min(x, axis, keepdims=True)
    return (x - min) / (max - min)


@log_calling
@tf.function
def get_accuracy(y_true, y_pred):
    output_dtype = y_pred.dtype
    y_true = tf.argmax(y_true, axis=-1)
    y_pred = tf.argmax(y_pred, axis=-1)
    return tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred), output_dtype))


class GlorotUniform(tf.keras.initializers.GlorotUniform):
    
    def __init__(self, scale=None, seed=None):
        super().__init__(seed)
        self.scale = 1 if scale is None else scale
    
    def __call__(self, *args, **kwargs):
        return self.scale * super().__call__(*args, **kwargs)


class MyLayer(tf.keras.layers.Layer):

    def __init__(self, network, dt, num_grids, **kwargs):
        super().__init__(**kwargs)
        self.network = network
        self.dt = dt
        self.num_grids = num_grids

        t0 = tf.constant(0.)
        self.tN = t0 + num_grids * dt

#         def pvf(t, x):
#             r"""
#             $x^{\prime} = \sigma\left(\sigma^{-1}(x) + \Delta t f(t, x; \theta) \right)$,
#             element-wisely.
#             """
#             return dsigmoid(inv_sigmoid(x)) * self.network(x)

        @log_calling
        @tf.function
        def pvf(t, x):
            with tf.GradientTape() as g:
                g.watch(x)
                f = self.network(x)
                r = rescale(x, axis=-1)
            return g.gradient(r, x, f)

        self._pvf = pvf
        self._node_fn = get_node_function(RKSolver(self.dt), 0., pvf)

    def call(self, x):
        y = self._node_fn(self.tN, x)
        return y


@log_calling
@tf.function
def process(X, y=None):
    X = tf.cast(X, tf.float32)
    X = X / 255.
    X = tf.reshape(X, [28 * 28])
    if y is None:
        return X
    else:
        y = tf.one_hot(y, 10)
        y = tf.cast(y, tf.float32)
        return X, y


def get_train_fn(model, optimizer,
                 num_epochs=5,
                 batch_size=128,
                 skip_step=50):
    loss_fn = tf.losses.CategoricalCrossentropy()

    @log_calling
    @tf.function
    def train_one_step(X, y):
        with tf.GradientTape() as tape:
            outputs = model(X)
            loss = loss_fn(y, outputs)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        accuracy = get_accuracy(y, outputs)
        return loss, accuracy

    @tf.function
    def train(X_train, y_train, X_test, y_test):
        num_steps_per_epoch = int(len(X_train) / batch_size)

        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        train_dataset = (train_dataset.map(process)
                          .shuffle(10000)
                          .repeat(num_epochs)
                          .batch(batch_size))
        test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
        test_dataset = test_dataset.map(process).batch(batch_size)

        @tf.function
        def evaluate():
            step = 0
            total_accuracy = 0.
            for X, y in test_dataset:
                outputs = model(X)
                total_accuracy += get_accuracy(y, outputs)
                step += 1
            return total_accuracy / tf.cast(step, total_accuracy.dtype)

        step = 0
        loss = float('inf')
        reg = float('inf')
        accuracy = 0.
        for X, y in train_dataset:
            loss, accuracy = train_one_step(X, y)

            if step % skip_step == 0:
                tf.print(step, loss, accuracy)

            if step % num_steps_per_epoch == 0:
                tf.print('testing')
                test_accuracy = evaluate()
                tf.print('test accuracy:', test_accuracy)

            step += 1
        return loss, accuracy

    return train


def clip(min, max, x):
    min = np.ones_like(x) * min
    max = np.ones_like(x) * max
    x = np.where(x < min, min, x)
    x = np.where(x > max, max, x)
    return x

In [2]:
input_dim = 28 * 28
network = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu',
                          kernel_initializer=GlorotUniform(0.1)),
    # tanh output for bounding the scale of phase vector field
    tf.keras.layers.Dense(input_dim, activation='tanh',
                          kernel_initializer=GlorotUniform(0.1)),
])
network.build([None, input_dim])

In [3]:
def get_datasets(raw_X, raw_y):
    involved_labels = {1, 3, 5}

    X, y = [], []
    for xi, yi in zip(raw_X, raw_y):
        if yi in involved_labels:
            X.append(xi)
            y.append(yi)
    return np.array(X), np.array(y)

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X_train, y_train = get_datasets(x_train, y_train)
X_test, y_test = get_datasets(x_test, y_test)

In [4]:
my_layer = MyLayer(network, dt=1e-1, num_grids=3)
output_layer = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])
model = tf.keras.Sequential([my_layer, output_layer])
model.build([None, 28 * 28])

optimizer = tf.compat.v1.train.AdamOptimizer()

train = get_train_fn(model, optimizer, num_epochs=10)
train(X_train, y_train, X_test, y_test)

testing
0 2.50347924 0
test accuracy: 0.0139973955
50 1.67159247 0.640625
100 1.37097931 0.6796875
testing
test accuracy: 0.672106
150 1.34138274 0.6171875
200 1.32991087 0.703125
250 1.20285666 0.6875
testing
test accuracy: 0.701038837
300 1.10391653 0.6171875
350 1.13243628 0.6953125
400 1.05768108 0.671875
testing
test accuracy: 0.699085712
450 1.01933336 0.7421875
500 1.04378784 0.734375
550 1.05227804 0.6875
testing
test accuracy: 0.715239286
600 0.934368432 0.6953125
650 1.0137589 0.7734375
700 0.930982113 0.6796875
testing
test accuracy: 0.702991962
750 0.94667387 0.7421875
800 0.88119173 0.6640625
850 0.860076308 0.6953125
testing
test accuracy: 0.743153572
900 0.893186808 0.703125
950 0.808332741 0.734375
testing
test accuracy: 0.749541461
1000 0.862797 0.75
1050 0.804313421 0.7265625
1100 0.684438 0.9375
testing
test accuracy: 0.95792383
1150 0.475499034 0.953125
1200 0.455723584 0.9453125
1250 0.451508105 0.953125
testing
test accuracy: 0.964108706
1300 0.427073389 0.984375


(<tf.Tensor: id=2643, shape=(), dtype=float32, numpy=0.42813936>,
 <tf.Tensor: id=2644, shape=(), dtype=float32, numpy=0.9285714>)

In [20]:
for num_grids in (1, 3, 5, 7, 10, 15, 20, 50, 100, 200, 500, 1000):
    my_layer_2 = MyLayer(network, dt=1e-1, num_grids=num_grids)
    model_2 = tf.keras.Sequential([my_layer_2, output_layer])
    model_2.build([None, 28 * 28])

    @tf.function
    def evaluate_2(X, y):
        batch_size = 64
        num_batches = int(len(X) / batch_size)
        dataset = tf.data.Dataset.from_tensor_slices((X, y))
        dataset = dataset.map(process).batch(batch_size)

        total_accuracy = 0.
        for xi, yi in dataset:
            outputs = model_2(xi)
            total_accuracy += get_accuracy(yi, outputs)
        return total_accuracy / tf.cast(num_batches, total_accuracy.dtype)

    print(f'num grids: {num_grids}, test accuracy: {evaluate_2(X_test, y_test).numpy()}')

num grids: 1, test accuracy: 0.983171284198761
num grids: 3, test accuracy: 0.9938095808029175
num grids: 5, test accuracy: 0.9837673902511597
num grids: 7, test accuracy: 0.94373619556427
num grids: 10, test accuracy: 0.8189998865127563
num grids: 15, test accuracy: 0.7044318914413452
num grids: 20, test accuracy: 0.6838201880455017
num grids: 50, test accuracy: 0.6801632046699524
num grids: 100, test accuracy: 0.4737710654735565
num grids: 200, test accuracy: 0.6788334250450134
num grids: 500, test accuracy: 0.31889674067497253
num grids: 1000, test accuracy: 0.35859546065330505


In [ ]:
from node.utils import tracer

def flip(array, ratio):
    is_flipped = np.random.random(size=array.shape) < ratio
    return np.where(is_flipped, 1 - array, array)

t0 = 0.
t1 = 1.
dt = 1e-2
traj_size = int((t1 - t0) / dt) + 1

n_data = 20
flip_ratio = 0.1

trace = tracer(RKSolver(1e-2), my_layer._pvf)
trajectories = trace(t0, t1, dt, data)
trajectories = tf.transpose(trajectories, [1, 0, 2])
trajectories = trajectories.numpy()


def get_trajectory(x):
    """Input shape `[28 * 28]`, output shape `[frames, 28, 28]`."""
    trajectory = trace(t0, t1, dt, [x]).numpy()[:,0,:]
    trajectory = np.reshape(trajectory, [28, 28])
    return trajectory

In [ ]:
i = 10
print(preds[i])
labels[i], np.argmax(preds[i])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation


def rescale(array):
    shape = array.shape
    y = np.reshape(array, [-1])
    y = (y - np.min(y)) / (np.max(y) - np.min(y))
    return np.reshape(y, shape)


def visualize_trajectory(trajectory):
    """
    Args:
        trajectory: np.array
            Shape `[frames, x_pixal, y_pixal]`.
    
    Returns: animation.FuncAnimation
    """
    fig = plt.figure()
    ax = plt.axes()
    img = ax.imshow(trajectory[0], cmap='gray')

    def init():
        img.set_data([[]])
        return img,

    def animate(i):
        y = rescale(trajectory[i])
        img.set_data(y)
        return img,

    anim = animation.FuncAnimation(
        fig, animate, init_func=init, frames=traj_size, blit=True)
    return anim


for i, trajectory in enumerate(trajectories):
    label = labels[i]
    anim = visualize_trajectory(trajectory.reshape([-1, 28, 28]))
    anim.save(f'../dat/trajectory/anim_i{i}_l{label}.mp4')
    plt.show()

## Conclusion

### Approach 1

~~1. Animation plotting shows that attractors exist for all the displayed instances.~~

~~1. By tracing the flipping ratio, we find that, while setting $\tilde{L} = 3$ in the training process, the flip ratio decreases from $\sim 0.1$ to $\sim 0.001$ only after $\tilde{L} > 30$ approximately for all trials. That is, the static phase vector field is trained without reaching the attractors. And when reaching the attractors, instances in the same class have little difference (but not vanishing), instances from different classes become evidently more distinct.~~

~~1. The attractors for the same class, even though close to each other, are far from single. It seems to confirm the conclusion in the study of Hebbian learning that high-dimensional dynamic systems have extremely many attractors.~~

1. After re-scaling by `lambda x: (x - min(x)) / (max(x) - min(x))`, there does exists attractors having the properties described above. So, it seems that the phase point flying straightly towords some direction specific for different classes. Or say, "attracted to the direction".

1. It seems that we encountered the chaos along the phase trajectory.

### Approach 2

$x^{\prime} = \sigma\left(\sigma^{-1}(x) + \Delta t f(t, x; \theta) \right)$

1. Attractors are reached at $L \sim 300$.
1. There are quite a lot of attractors.
1. Indeed, in this case, the $x^{\alpha} = 0, 1$ for $\forall \alpha$ are attractors.
1. However, this approach introduces an artificial area of fixed points (i.e. $x^{\alpha} = 0, 1$ for $\forall \alpha$), which is not what we hope for.

### Approach 3

$x^{\prime} = r\left(x + \Delta t f(t, x; \theta) \right)$
where $r^{\alpha}(x) := \left( x^{\alpha} - min(x) \right) / \left( max(x) - min(x) \right)$, and $x$ is initialized s.t. $min(x) = 0$ and $max(x) = 1$. The max and min functions are in soft version.

## References:

1. [Chaos appears in RGE (as a high dimensional non-linear ODE)](https://physics.stackexchange.com/a/55057) (the [paper](https://arxiv.org/abs/hep-th/0304178) related).